In [105]:
path1='C:\\Users\\Korisnik\\Desktop\\diplomska\\Version2\\Dataset\\Subject1\\Activity11\\Trial2\\Subject1Activity11Trial2.csv'
path2='C:\\Users\\Korisnik\\Desktop\\diplomska\\Version2\\Dataset\\Subject1\\Activity11\\Trial2\\'
# extract_features(path1,path2,'Subject1','Activity11','Trial2')

In [16]:
#ankle, belt, neck, wrist
def getSensors():
    stuff =['Ankle','Belt','Neck','Wrist']
    sensorList=[]
    for L in range(1, len(stuff)+1):
        for subset in itertools.combinations(stuff, L):
            p=[]
            for i in subset:
                p.append(i+'Accelerometer: x-axis (g)')
                p.append(i+'Accelerometer: y-axis (g)')
                p.append(i+'Accelerometer: z-axis (g)')
            sensorList.append(p)
   
    return sensorList

def getFeatures():
    ftList = ['Mean',
        'StandardDeviation',
        'RootMeanSquare',
        'MaximalAmplitude',
        'MinimalAmplitude',
        'Median',
        'Number of zero-crossing',
       'Skewness',
        'Kurtosis',
        'First Quartile',
        'Third Quartile',
        'Autocorrelation'
        ] 
    return ftList
    

In [56]:
sensorList=getSensors()[5]
print(sensorList)

['AnkleAccelerometer: x-axis (g)', 'AnkleAccelerometer: y-axis (g)', 'AnkleAccelerometer: z-axis (g)', 'NeckAccelerometer: x-axis (g)', 'NeckAccelerometer: y-axis (g)', 'NeckAccelerometer: z-axis (g)']


In [24]:
def getHeader(sensors, features):
    final=[[]]
    final[0].append('Timestamp')
    for s in sensors:
        for f in features:
            final[0].append(s+f)
            
    final[0].append('Subject')
    final[0].append('Activity')
    final[0].append('Trial')
    final[0].append('Tag')
    return final

In [25]:
import numpy as np
from scipy.stats import kurtosis, skew, entropy
import pytest
from scipy.fftpack import rfft, rfftfreq
from datetime import datetime as dt
def getFeature(inpt,ftList):
    output = [] #the array we'll be returning:
    #[mean,standardDeviation,rootMeanSquare,maxAmp,minAmp,median,numZero-cross,
    #skewness,kurtosis,Q1,Q3,autocorrelation]
    rms = 0 #root mean square
    amp = [abs(inpt[i]) for i in range(0,len(inpt))] #to store amplitudes
    #variables used to count when zero is crossed
    bn = 1
    nzc = 0
    bnd = False
    bnF = False
    for i in range(0, len(inpt)):
        rms = rms + (inpt[i]**2)
        #this is to see when zero is being crossed:
        if (inpt[i] != 0):
            bnd = True
            if(bnF == False):
                bnF = True
                bn = inpt[i]/abs(inpt[i])
        else:
            bnd = False
        if (bnd):
            sgn = inpt[i]/abs(inpt[i])
            if (sgn != bn):
                nzc = nzc + 1
            bn = inpt[i]/abs(inpt[i])
    for feature in ftList:
        if feature == 'Mean':
            output.append(np.mean(inpt))
        elif feature == 'StandardDeviation':
            output.append(np.std(inpt))
        elif feature == 'RootMeanSquare':
            output.append((np.sqrt(rms) / np.sqrt(len(inpt))))
        elif feature == 'MaximalAmplitude':
            output.append(max(amp))
        elif feature == 'MinimalAmplitude':
            output.append(min(amp))
        elif feature == 'Median':
            output.append(np.median(inpt))
        elif feature == 'Number of zero-crossing':
            output.append(nzc)
        elif feature == 'Skewness':
            output.append(skew(inpt))
        elif feature == 'Kurtosis':
            output.append(kurtosis(inpt))
        elif feature == 'First Quartile':
            output.append(np.percentile(inpt,25,interpolation = 'midpoint'))
        elif feature == 'Third Quartile':
            output.append(np.percentile(inpt,75,interpolation = 'midpoint'))
        elif feature == 'Autocorrelation':
            kt = np.correlate(inpt,inpt,mode='full')
            output.append(np.median(kt))
    return output


    

In [50]:
import statistics
def find_max_mode(list1):
    list_table = statistics._counts(list1)
    len_table = len(list_table)

    if len_table == 1:
        max_mode = statistics.mode(list1)
    else:
        new_list = []
        for i in range(len_table):
            new_list.append(list_table[i][0])
        max_mode = max(new_list) # use the max value here
    return max_mode

In [57]:
import csv
import datetime
from statistics import mode
import os
#Function used to extract and return time from sepcific row        
def getTime(row):
    year = int(row[0][0:4])
    month = int(row[0][5:7])
    day = int(row[0][8:10])
    hour = int(row[0][11:13])
    minute = int(row[0][14:16])
    second = int(row[0][17:19])
    microsecond = int(row[0][20:26])
    return datetime.datetime(year,month,day,hour,minute,second,microsecond)

def extract_features(path1,path2,sub,act,trl,index_array, name,index):
    win_len1=1
    win_len2=0.5
    #open csv file
    csvFile=open(path1)
    csvReader=csv.reader(csvFile,delimiter=',')
    csvArray=[]
    line_count=0
    for row in csvReader:
        if line_count==0 or line_count==1:
            line_count+=1
        else:
            pom=[]
            pom.append(row[0])
            for i in range(0,len(index_array)):
                for k in range(index_array[i][0],index_array[i][1]):
                    pom.append(row[k])
            pom.append(row[43])
            pom.append(row[44])
            pom.append(row[45])
            pom.append(row[46])
            csvArray.append(pom)
    csvFile.close()
    csvArray = [[item.strip('\n') for item in array]for array in csvArray]
    start_time=getTime(csvArray[0])
    end_time=getTime(csvArray[len(csvArray)-1])
    
    sensorList=getSensors()[index]
    featureList=getFeatures()
    final=getHeader(sensorList,featureList)
    temp=[]
    subSensors=[]
    j=1
    while start_time+datetime.timedelta(seconds=win_len1)<=end_time:
        for row in csvArray:
            if start_time<=getTime(row) and getTime(row)<=start_time+datetime.timedelta(seconds=win_len1):
                temp.append(row)
        try:
            for i in range(len(temp[0])):
    #             print(temp[i])
                subSensors.append([])
                for row in temp:
                    subSensors[i].append(row[i])
    #         print(subSensors)      
            final.append([])
            final[j].append(datetime.datetime.strftime(start_time, '%Y-%m-%dT%H:%M:%S.%f'))
            for row in subSensors[1:len(sensorList)+1]:
                nrow=list(map(float,row))
                features=getFeature(nrow, featureList)


                for f in features:
                    final[j].append(f)
        
        except Exception as ex:
            if str(ex) == 'list index out of range':
                print('------Possible data absence between timestamps: ')
                nexttime = start_time + datetime.timedelta(seconds = win_len2)
                print('-----------' +str(start_time)+ ' and ' + str(nexttime))
            else:
                print('---Unexpected error: ' + str(ex))
            
            
        tag=[]
        sub_niza=[]
        act_niza=[]
        trl_niza=[]
        for row in temp:
            tag.append(row[len(row)-1])
            trl_niza.append(row[len(row)-2])
            act_niza.append(row[len(row)-3])
            sub_niza.append(row[len(row)-4])
            
        final[j].append(find_max_mode(sub_niza))
        final[j].append(find_max_mode(act_niza))
        final[j].append(find_max_mode(trl_niza))
        final[j].append(find_max_mode(tag))
        j+=1
        temp=[]
        subSensors=[]
        start_time+=datetime.timedelta(seconds=win_len2)
        
    try:
        if not os.path.exists(path2):
            os.makedirs(path2)
    except OSError:
        print ('An error ocurred while creating direcory: "' +  path2 +'"')

    with open (path2 + sub + act + trl + 'Features_'+name + '.csv', 'w') as newDataSet:
        csvwriter = csv.writer(newDataSet,lineterminator = '\n')
        for row in final:
            csvwriter.writerow(row)
        newDataSet.close()
            
        
        
        
        
    
            
            
            
    
    

In [35]:
index=[[1,4],[15,18],[22,25],[29,32]]
stuff =['Ankle','Belt','Neck','Wrist']
iterate=[]
names=[]
for L in range(1, len(index)+1):
    for subset in itertools.combinations(index, L):
        p=[]
        for i in subset:
            p.append(i)
        iterate.append(p)
for L in range(1, len(stuff)+1):
    for subset in itertools.combinations(stuff, L):
        p=''
        for i in subset:
            p+=i
        names.append(p)

In [49]:
print(iterate[10])
for i in range(0,len(iterate[10])):
    for k in range(iterate[10][i][0],iterate[10][i][1]):
        print(k)
        

[[1, 4], [15, 18], [22, 25]]
1
2
3
15
16
17
22
23
24


In [60]:
import csv

d_base_path="C:\\Users\\Korisnik\\Desktop\\diplomska\\Version4\\Dataset\\"
n_sub=[1,17]
n_act=[1,11]
n_trl=[1,3]

for p in range(0,len(iterate)):
    print('Iteration ',p,' out of 15')
    for i in range(n_sub[0],n_sub[1]+1):
        sub='Subject'+str(i)
        for j in range(n_act[0],n_act[1]+1):
            act='Activity'+str(j)
            for k in range(n_trl[0],n_trl[1]+1):
                if (i == 1 and j == 11 and (k ==2 or k==3)) or (i==5 and j==7 and k==1) or(i==5 and j==8 and k==2) or(i==8 and j==10 and k==2)or (i==8 and j==11 and (k==2 or k==3))or(i==12 and j==7and k==2)or(i==14 and j==7 and k==2):
                    print("ok")
                else:
                    trl='Trial'+str(k)
                    path1=d_base_path+sub+'\\'+act+'\\'+trl+'\\'+sub+act+trl+'.csv'
                    path2=d_base_path+sub+'\\'+act+'\\'+trl+'\\'
                    extract_features(path1,path2,sub,act,trl,iterate[p],names[p],p)
                    print(trl+'...Done')
            print(act+'..Done')

        print(sub+'... Done')

            

Iteration  0  out of 15
Trial1...Done
Trial2...Done
Trial3...Done
Activity1..Done
Trial1...Done
Trial2...Done
Trial3...Done
Activity2..Done
Trial1...Done
Trial2...Done
Trial3...Done
Activity3..Done
Trial1...Done
Trial2...Done
Trial3...Done
Activity4..Done
Trial1...Done
Trial2...Done
Trial3...Done
Activity5..Done
Trial1...Done
Trial2...Done
Trial3...Done
Activity6..Done
Trial1...Done
Trial2...Done
Trial3...Done
Activity7..Done
Trial1...Done
Trial2...Done
Trial3...Done
Activity8..Done
Trial1...Done
Trial2...Done
Trial3...Done
Activity9..Done
Trial1...Done
Trial2...Done
Trial3...Done
Activity10..Done
Trial1...Done
ok
ok
Activity11..Done
Subject1... Done
Trial1...Done
Trial2...Done
Trial3...Done
Activity1..Done
Trial1...Done
Trial2...Done
Trial3...Done
Activity2..Done
Trial1...Done
Trial2...Done
Trial3...Done
Activity3..Done
Trial1...Done
Trial2...Done
Trial3...Done
Activity4..Done
Trial1...Done
Trial2...Done
Trial3...Done
Activity5..Done
Trial1...Done
Trial2...Done
Trial3...Done
Activity6

Trial3...Done
Activity7..Done
Trial1...Done
Trial2...Done
Trial3...Done
Activity8..Done
Trial1...Done
Trial2...Done
Trial3...Done
Activity9..Done
Trial1...Done
Trial2...Done
Trial3...Done
Activity10..Done
Trial1...Done
Trial2...Done
Trial3...Done
Activity11..Done
Subject13... Done
Trial1...Done
Trial2...Done
Trial3...Done
Activity1..Done
Trial1...Done
Trial2...Done
Trial3...Done
Activity2..Done
Trial1...Done
Trial2...Done
Trial3...Done
Activity3..Done
Trial1...Done
Trial2...Done
Trial3...Done
Activity4..Done
Trial1...Done
Trial2...Done
Trial3...Done
Activity5..Done
Trial1...Done
Trial2...Done
Trial3...Done
Activity6..Done
Trial1...Done
ok
Trial3...Done
Activity7..Done
Trial1...Done
Trial2...Done
Trial3...Done
Activity8..Done
Trial1...Done
Trial2...Done
Trial3...Done
Activity9..Done
Trial1...Done
Trial2...Done
Trial3...Done
Activity10..Done
Trial1...Done
Trial2...Done
Trial3...Done
Activity11..Done
Subject14... Done
Trial1...Done
Trial2...Done
Trial3...Done
Activity1..Done
Trial1...Done

Trial1...Done
Trial2...Done
Trial3...Done
Activity3..Done
Trial1...Done
Trial2...Done
Trial3...Done
Activity4..Done
Trial1...Done
Trial2...Done
Trial3...Done
Activity5..Done
Trial1...Done
Trial2...Done
Trial3...Done
Activity6..Done
Trial1...Done
Trial2...Done
Trial3...Done
Activity7..Done
Trial1...Done
Trial2...Done
Trial3...Done
Activity8..Done
Trial1...Done
Trial2...Done
Trial3...Done
Activity9..Done
Trial1...Done
Trial2...Done
Trial3...Done
Activity10..Done
Trial1...Done
Trial2...Done
Trial3...Done
Activity11..Done
Subject9... Done
Trial1...Done
Trial2...Done
Trial3...Done
Activity1..Done
Trial1...Done
Trial2...Done
Trial3...Done
Activity2..Done
Trial1...Done
Trial2...Done
Trial3...Done
Activity3..Done
Trial1...Done
Trial2...Done
Trial3...Done
Activity4..Done
Trial1...Done
Trial2...Done
Trial3...Done
Activity5..Done
Trial1...Done
Trial2...Done
Trial3...Done
Activity6..Done
Trial1...Done
Trial2...Done
Trial3...Done
Activity7..Done
Trial1...Done
Trial2...Done
Trial3...Done
Activity8..

Trial1...Done
Trial2...Done
Trial3...Done
Activity9..Done
Trial1...Done
Trial2...Done
Trial3...Done
Activity10..Done
Trial1...Done
Trial2...Done
Trial3...Done
Activity11..Done
Subject4... Done
Trial1...Done
Trial2...Done
Trial3...Done
Activity1..Done
Trial1...Done
Trial2...Done
Trial3...Done
Activity2..Done
Trial1...Done
Trial2...Done
Trial3...Done
Activity3..Done
Trial1...Done
Trial2...Done
Trial3...Done
Activity4..Done
Trial1...Done
Trial2...Done
Trial3...Done
Activity5..Done
Trial1...Done
Trial2...Done
Trial3...Done
Activity6..Done
ok
Trial2...Done
Trial3...Done
Activity7..Done
Trial1...Done
ok
Trial3...Done
Activity8..Done
Trial1...Done
Trial2...Done
Trial3...Done
Activity9..Done
Trial1...Done
Trial2...Done
Trial3...Done
Activity10..Done
Trial1...Done
Trial2...Done
Trial3...Done
Activity11..Done
Subject5... Done
Trial1...Done
Trial2...Done
Trial3...Done
Activity1..Done
Trial1...Done
Trial2...Done
Trial3...Done
Activity2..Done
Trial1...Done
Trial2...Done
Trial3...Done
Activity3..Don

Trial2...Done
Trial3...Done
Activity4..Done
Trial1...Done
Trial2...Done
Trial3...Done
Activity5..Done
Trial1...Done
Trial2...Done
Trial3...Done
Activity6..Done
Trial1...Done
Trial2...Done
Trial3...Done
Activity7..Done
Trial1...Done
Trial2...Done
Trial3...Done
Activity8..Done
Trial1...Done
Trial2...Done
Trial3...Done
Activity9..Done
Trial1...Done
Trial2...Done
Trial3...Done
Activity10..Done
Trial1...Done
Trial2...Done
Trial3...Done
Activity11..Done
Subject17... Done
Iteration  3  out of 15
Trial1...Done
Trial2...Done
Trial3...Done
Activity1..Done
Trial1...Done
Trial2...Done
Trial3...Done
Activity2..Done
Trial1...Done
Trial2...Done
Trial3...Done
Activity3..Done
Trial1...Done
Trial2...Done
Trial3...Done
Activity4..Done
Trial1...Done
Trial2...Done
Trial3...Done
Activity5..Done
Trial1...Done
Trial2...Done
Trial3...Done
Activity6..Done
Trial1...Done
Trial2...Done
Trial3...Done
Activity7..Done
Trial1...Done
Trial2...Done
Trial3...Done
Activity8..Done
Trial1...Done
Trial2...Done
Trial3...Done


...

Trial2...Done
Trial3...Done
Activity5..Done
Trial1...Done
Trial2...Done
Trial3...Done
Activity6..Done
Trial1...Done
Trial2...Done
Trial3...Done
Activity7..Done
Trial1...Done
Trial2...Done
Trial3...Done
Activity8..Done
Trial1...Done
Trial2...Done
Trial3...Done
Activity9..Done
Trial1...Done
Trial2...Done
Trial3...Done
Activity10..Done
Trial1...Done
Trial2...Done
Trial3...Done
Activity11..Done
Subject16... Done
Trial1...Done
Trial2...Done
Trial3...Done
Activity1..Done
Trial1...Done
Trial2...Done
Trial3...Done
Activity2..Done
Trial1...Done
Trial2...Done
Trial3...Done
Activity3..Done
Trial1...Done
Trial2...Done
Trial3...Done
Activity4..Done
Trial1...Done
Trial2...Done
Trial3...Done
Activity5..Done
Trial1...Done
Trial2...Done
Trial3...Done
Activity6..Done
Trial1...Done
Trial2...Done
Trial3...Done
Activity7..Done
Trial1...Done
Trial2...Done
Trial3...Done
Activity8..Done
Trial1...Done
Trial2...Done
Trial3...Done
Activity9..Done
Trial1...Done
Trial2...Done
Trial3...Done
Activity10..Done
Trial1.

Trial3...Done
Activity11..Done
Subject11... Done
Trial1...Done
Trial2...Done
Trial3...Done
Activity1..Done
Trial1...Done
Trial2...Done
Trial3...Done
Activity2..Done
Trial1...Done
Trial2...Done
Trial3...Done
Activity3..Done
Trial1...Done
Trial2...Done
Trial3...Done
Activity4..Done
Trial1...Done
Trial2...Done
Trial3...Done
Activity5..Done
Trial1...Done
Trial2...Done
Trial3...Done
Activity6..Done
Trial1...Done
ok
Trial3...Done
Activity7..Done
Trial1...Done
Trial2...Done
Trial3...Done
Activity8..Done
Trial1...Done
Trial2...Done
Trial3...Done
Activity9..Done
Trial1...Done
Trial2...Done
Trial3...Done
Activity10..Done
Trial1...Done
Trial2...Done
Trial3...Done
Activity11..Done
Subject12... Done
Trial1...Done
Trial2...Done
Trial3...Done
Activity1..Done
Trial1...Done
Trial2...Done
Trial3...Done
Activity2..Done
Trial1...Done
Trial2...Done
Trial3...Done
Activity3..Done
Trial1...Done
Trial2...Done
Trial3...Done
Activity4..Done
Trial1...Done
Trial2...Done
Trial3...Done
Activity5..Done
Trial1...Done


Trial2...Done
Trial3...Done
Activity6..Done
Trial1...Done
Trial2...Done
Trial3...Done
Activity7..Done
Trial1...Done
Trial2...Done
Trial3...Done
Activity8..Done
Trial1...Done
Trial2...Done
Trial3...Done
Activity9..Done
Trial1...Done
Trial2...Done
Trial3...Done
Activity10..Done
Trial1...Done
Trial2...Done
Trial3...Done
Activity11..Done
Subject7... Done
Trial1...Done
Trial2...Done
Trial3...Done
Activity1..Done
Trial1...Done
Trial2...Done
Trial3...Done
Activity2..Done
Trial1...Done
Trial2...Done
Trial3...Done
Activity3..Done
Trial1...Done
Trial2...Done
Trial3...Done
Activity4..Done
Trial1...Done
Trial2...Done
Trial3...Done
Activity5..Done
Trial1...Done
Trial2...Done
Trial3...Done
Activity6..Done
Trial1...Done
Trial2...Done
Trial3...Done
Activity7..Done
Trial1...Done
Trial2...Done
Trial3...Done
Activity8..Done
Trial1...Done
Trial2...Done
Trial3...Done
Activity9..Done
Trial1...Done
ok
Trial3...Done
Activity10..Done
Trial1...Done
ok
ok
Activity11..Done
Subject8... Done
Trial1...Done
Trial2...

Trial1...Done
Trial2...Done
Trial3...Done
Activity1..Done
Trial1...Done
Trial2...Done
Trial3...Done
Activity2..Done
Trial1...Done
Trial2...Done
Trial3...Done
Activity3..Done
Trial1...Done
Trial2...Done
Trial3...Done
Activity4..Done
Trial1...Done
Trial2...Done
Trial3...Done
Activity5..Done
Trial1...Done
Trial2...Done
Trial3...Done
Activity6..Done
Trial1...Done
Trial2...Done
Trial3...Done
Activity7..Done
Trial1...Done
Trial2...Done
Trial3...Done
Activity8..Done
Trial1...Done
Trial2...Done
Trial3...Done
Activity9..Done
Trial1...Done
Trial2...Done
Trial3...Done
Activity10..Done
Trial1...Done
Trial2...Done
Trial3...Done
Activity11..Done
Subject3... Done
Trial1...Done
Trial2...Done
Trial3...Done
Activity1..Done
Trial1...Done
Trial2...Done
Trial3...Done
Activity2..Done
Trial1...Done
Trial2...Done
Trial3...Done
Activity3..Done
Trial1...Done
Trial2...Done
Trial3...Done
Activity4..Done
Trial1...Done
Trial2...Done
Trial3...Done
Activity5..Done
Trial1...Done
Trial2...Done
Trial3...Done
Activity6..

Trial1...Done
Trial2...Done
Trial3...Done
Activity11..Done
Subject15... Done
Trial1...Done
Trial2...Done
Trial3...Done
Activity1..Done
Trial1...Done
Trial2...Done
Trial3...Done
Activity2..Done
Trial1...Done
Trial2...Done
Trial3...Done
Activity3..Done
Trial1...Done
Trial2...Done
Trial3...Done
Activity4..Done
Trial1...Done
Trial2...Done
Trial3...Done
Activity5..Done
Trial1...Done
Trial2...Done
Trial3...Done
Activity6..Done
Trial1...Done
Trial2...Done
Trial3...Done
Activity7..Done
Trial1...Done
Trial2...Done
Trial3...Done
Activity8..Done
Trial1...Done
Trial2...Done
Trial3...Done
Activity9..Done
Trial1...Done
Trial2...Done
Trial3...Done
Activity10..Done
Trial1...Done
Trial2...Done
Trial3...Done
Activity11..Done
Subject16... Done
Trial1...Done
Trial2...Done
Trial3...Done
Activity1..Done
Trial1...Done
Trial2...Done
Trial3...Done
Activity2..Done
Trial1...Done
Trial2...Done
Trial3...Done
Activity3..Done
Trial1...Done
Trial2...Done
Trial3...Done
Activity4..Done
Trial1...Done
Trial2...Done
Trial

Join data for every Subject

In [62]:
stuff =['Ankle','Belt','Neck','Wrist']
names=[]
for L in range(1, len(stuff)+1):
    for subset in itertools.combinations(stuff, L):
        p=''
        for i in subset:
            p+=i
        names.append(p)
print(names)

['Ankle', 'Belt', 'Neck', 'Wrist', 'AnkleBelt', 'AnkleNeck', 'AnkleWrist', 'BeltNeck', 'BeltWrist', 'NeckWrist', 'AnkleBeltNeck', 'AnkleBeltWrist', 'AnkleNeckWrist', 'BeltNeckWrist', 'AnkleBeltNeckWrist']


In [68]:
def featureJoiner(path,
                  path1,
                  sub,i,
                  n_act=[1,11],
                  n_trl=[1,3],
                  name='Ankle'):
    f_row = 1
    f_name = 'Features'  + '.csv'
    f_flg = True
    w = open(path1 + f_name, 'w')
    try:
        for j in range(n_act[0],n_act[1]+1):
            act = 'Activity' + str(j)
            for k in range(n_trl[0],n_trl[1]+1):
                if (i == 1 and j == 11 and (k ==2 or k==3)) or (i==5 and j==7 and k==1) or(i==5 and j==8 and k==2) or(i==8 and j==10 and k==2)or (i==8 and j==11 and (k==2 or k==3))or(i==12 and j==7and k==2)or(i==14 and j==7 and k==2):
                    print("ok")
                else:
                    trl = 'Trial' + str(k)
                    f_path = path + '\\' + act + '\\' + trl + '\\'
                    r = open(f_path +sub+act+trl+'Features_'+str(name)+'.csv','r')
                    txt = r.read()
                    r.close()
                    file = txt.split('\n')
                    start = f_row
                    if f_flg:
                        w.write(file[0])
                        if f_row == 2:
                            w.write('\n' + file[1])
                        f_flg = False
                    for row in file[start:]:

                        q = row.split(',')
                        if (q[0]!='') and (q[-1]!=''):
                            w.write('\n'+row)
    except Exception as e:
        print('--An unexpected error ocurred while writing ' + f_name +' :')
        print('-----' + str(e))
    w.close()

In [69]:
def createFolder(path):
    try:
        if not os.path.exists(path):
            os.makedirs(path)
    except:
        print("An error ocurred while creatig directory: "+ path)

In [71]:
n_sub=[1,17]
for n in names:
    path='C:\\Users\\Korisnik\\Desktop\\diplomska\\Version4\\'+n
    createFolder(path)
    for i in range(n_sub[0],n_sub[1]+1):
        path1='C:\\Users\\Korisnik\\Desktop\\diplomska\\Version4\\Dataset\\Subject'+str(i)
        path2=path+'\\Subject'+str(i)
        featureJoiner(path1,path2,'Subject'+str(i),i,
                      n_act=[1,11],
                      n_trl=[1,3],
                      name=n)
    
print('Done')


ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
Done


In [2]:
path='C:\\Users\\Korisnik\\Desktop\\diplomska\\Version4\\Dataset\\Subject1\\Activity6\\Trial1\\Subject1Activity6Trial1.csv'

import pandas as pd
df=pd.read_csv(path)
df.head(20)

ParserError: Error tokenizing data. C error: Expected 46 fields in line 3, saw 47
